In [ ]:
#Script to populate pollutant concentrations on polutant release dataset
import pandas as pd
import time
start = time.process_time()
facilities = pd.read_csv('E:\Research\Datasets\Pollutant emissions data\E-PRTR_database_v16_csv\dbo.PUBLISH_FACILITYREPORT.csv', encoding = "ISO-8859-1")
facilities_nearby_stations = facilities_nearby_stations.dropna()
facilities_nearby_stations = facilities_nearby_stations[(facilities_nearby_stations[['Dist_CO', 'Dist_O3', 'Dist_PM10', 'Dist_PM2.5', 'Dist_SO2']] < 50).all(1)]
pollutant_release = pollutant_release.loc[pollutant_release['FacilityID'].isin(facilities_nearby_stations['FacilityID'].unique())]
#Add new columns for pollutants
pollutant_release = pd.concat([pollutant_release, pd.DataFrame(columns=["SO2", "C6H6", "CO", "PM10", "PM2.5", "O3"])], sort=False)

#Function to populate air quality pollutant concentration
def populateAveragePolutantReading(row):
    facilityId = row['FacilityID']
    polutants = ["SO2", "C6H6", "CO", "PM10", "PM2.5", "O3"]
    #For each pollutant and year, populate concentration
    for pol in polutants:
        reportingYearStations = facilities_nearby_stations.loc[(facilities_nearby_stations['FacilityID'] == facilityId) & (facilities_nearby_stations['Year'] == row['ReportingYear'])][pol]
        if reportingYearStations.shape[0] > 0:
            airQualitypollutants = airQualityStationsMeanReadings.loc[(airQualityStationsMeanReadings['AirQualityStationEoICode'] == reportingYearStations.values[0]) & (airQualityStationsMeanReadings['Year'] == row['ReportingYear'])]
            if airQualitypollutants.shape[0] > 0:
                row[pol] = airQualitypollutants[pol].values[0]
    return row

#populate for each facility
pollutant_release_merged = pollutant_release.apply(lambda x: populateAveragePolutantReading(x), axis = 1)

#Convert to csv
pollutant_release_merged.to_csv("E:/Research/Processing Datasets/Pollutant_Release_Impact.csv", index = False)
print("Completed second merge in ", time.process_time() - start)

In [ ]:
#scripts to convert each pollutant group to a column 
pollutant_release_merged = pd.read_csv('E:/Research/Processing Datasets/Pollutant_Release_Impact.csv')
groupedPollutantRelease = pollutant_release_merged.groupby(['FacilityID', 'ReportingYear'], as_index=False).agg({'SO2': 'mean', 'CO': 'mean', 'PM10': 'mean', 'O3': 'mean',  'PM2.5': 'mean'})

def populatePollutantRelease(row):
    
    subset = pollutant_release_merged[(pollutant_release_merged['FacilityID'] == row['FacilityID']) & (pollutant_release_merged['ReportingYear'] == row['ReportingYear'])]
    for releasedpol in subset['PollutantGroupCode'].unique():
        row[releasedpol] = sum(subset[subset['PollutantGroupCode'] == releasedpol]['TotalQuantity'].values)
    return row
groupedPollutantRelease = groupedPollutantRelease.apply(lambda x: populatePollutantRelease(x), axis = 1)
groupedPollutantRelease = pd.merge(groupedPollutantRelease,facilities[['FacilityID', 'CountryCode', 'CountryName']].drop_duplicates(), on = 'FacilityID')

In [ ]:
#Merge pollutant releases based on city
cityPopulatedFacilitiesDataset = facis[(facis['FacilityID'].isin(t2mp['FacilityID'].unique())) & (facis['City'].notna())]
cityPopulatedFacilitiesDataset = cityPopulatedFacilitiesDataset.drop_duplicates('FacilityID')
newmerge = pd.merge(t2mp, cityPopulatedFacilitiesDataset[['FacilityID', 'City']].drop_duplicates('FacilityID'), on='FacilityID')
cityWiseMerged = newmerge.groupby(['City', 'CountryCode', 'ReportingYear']).agg({'CHLORG':'sum', 'GRHGAS':'sum', 'HEVMET':'sum', 'INORG':'sum', 'OTHGAS':'sum', 'OTHORG':'sum', 'PEST':'sum', 'SO2': 'mean', 'CO': 'mean', 'PM10': 'mean', 'O3': 'mean', 'PM2.5': 'mean'}).reset_index()
filteredNonZeroData = cityWiseMerged[(cityWiseMerged['GRHGAS']>0) & (cityWiseMerged['HEVMET']>0) & (cityWiseMerged['OTHGAS']>0)].dropna()
filteredNonZeroData.to_csv('cityWiseMergedFiftyKms.csv', index = False)